In [1]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=720):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "010620"
    company_name = "HD현대미포"

    # 데이터 수집
    df = get_day_price(code, max_pages=720)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜       종가   전일비       시가       고가       저가     거래량
2025.04.25 163500.0  5000 158600.0 164700.0 158500.0  743766
2025.04.24 158500.0 20800 137300.0 158500.0 136200.0 1731192
2025.04.23 137700.0  4100 135000.0 139100.0 132100.0  387758
2025.04.22 133600.0  3600 128000.0 134500.0 128000.0  326287
2025.04.21 130000.0 -4200 131400.0 132000.0 126000.0  485705
2025.04.18 134200.0  -800 137600.0 137900.0 130800.0  290761
2025.04.17 135000.0  3100 132900.0 135500.0 131100.0  305130
2025.04.16 131900.0 -1400 132700.0 134300.0 130800.0  277192
2025.04.15 133300.0  1200 131000.0 133800.0 130400.0  234847
2025.04.14 132100.0  2600 133600.0 133600.0 129100.0  493164
2025.04.11 129500.0  6300 125200.0 129900.0 125000.0 537715
2025.04.10 123200.0  7300 122000.0 124100.0 119700.0 524429
2025.04.09 115900.0 -2300 121700.0 122000.0 114500.0 447795
2025.04.08 118200.0  7400 115100.0 119100.0 113900.0 366130
2025.04.07 110800.0 -5300 112100.0 113000.0 108800.0 388142
2025.04.04 116100.0 -1300 116